In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from graphviz import Digraph

In [2]:
class Node:
    def __init__(self, attribute=None, value=None, children=None, label=None):
        self.attribute = attribute
        self.value = value
        self.children = children if children else {}
        self.label = label

In [3]:
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None

    def _best_split(self, X, y):
        best_split = {}
        best_gini = 1.0
        n_rows, n_cols = X.shape
        for f_idx in range(n_cols):
            X_curr = X[:, f_idx]
            thresholds = np.unique(X_curr)
            for threshold in thresholds:
                df_left = np.concatenate((X[X_curr <= threshold], y[X_curr <= threshold].reshape(-1, 1)), axis=1)
                df_right = np.concatenate((X[X_curr > threshold], y[X_curr > threshold].reshape(-1, 1)), axis=1)
                if (len(df_left) > 0 and len(df_right) > 0):
                    y = np.concatenate((X, y.reshape(-1, 1)), axis=1)[:, -1]
                    gini = self._gini_index(X, y, f_idx, threshold)
                    if gini < best_gini:
                        best_gini = gini
                        best_split = {
                            'feature_index': f_idx,
                            'threshold': threshold,
                            'df_left': df_left,
                            'df_right': df_right,
                            'gini': gini
                        }
        return best_split


    def _build(self, X, y, depth=0):
        n_rows, n_cols = X.shape
        if n_rows >= self.min_samples_split and depth <= self.max_depth:
            best = self._best_split(X, y)
            if best['gini'] > 0:
                children = []
                for data, side in [(best['df_left'], 'left'), (best['df_right'], 'right')]:
                    child = self._build(
                        X=data[:, :-1],
                        y=data[:, -1],
                        depth=depth + 1
                    )
                    children.append((side, child))
                return Node(
                    feature=best['feature_index'],
                    threshold=best['threshold'],
                    children=children,
                    gini=best['gini']
                )
        return Node(
            value=Counter(y).most_common(1)[0][0]
        )

    def _gini_impurity(self, y):
        _,counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        gini = 1 - np.sum(probabilities ** 2)
        return gini


    def _gini_index(self, X, y, feature_index, threshold):
        left_indices = X[:, feature_index] <= threshold
        right_indices = X[:, feature_index] > threshold

        left_gini = self._gini_impurity( y[left_indices])
        right_gini = self._gini_impurity(y[right_indices])

        num_left = len(y[left_indices])
        num_right = len(y[right_indices])

        gini_index = (len(y[left_indices]) / (len(y[left_indices])+len(y[left_indices]))) * left_gini 
        + (len(y[right_indices]) /(len(y[left_indices])+len(y[right_indices]))) * right_gini
        return gini_index


    def fit(self, X, y):
        self.root = self._build(X, y)

    def _predict(self, x, tree):
        if tree.value is not None:
            return tree.value
        feature_value = x[tree.feature]
        for side, child in tree.children:
            if side == 'left' and feature_value <= tree.threshold:
                return self._predict(x, child)
            elif side == 'right' and feature_value > tree.threshold:
                return self._predict(x, child)

    def predict(self, X):
        return [self._predict(x, self.root) for x in X]


In [4]:
class Node:
    def __init__(self, feature=None, threshold=None, children=None, value=None, gain=None):
        self.feature = feature
        self.threshold = threshold
        self.children = children
        self.value = value
        self.gain = gain

class DecisionTree2:
    def __init__(self, min_samples_split=2, max_depth=100):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None

    def _information_gain(self, parent , left , right):
        def entropy(y):
            counts = np.bincount(np.array(y , dtype=np.int64))
            percent = counts / len(y)
            entropy_value =0
            for p in percent:
                if p>0:
                    entropy_value += -(p*np.log2(p))
            return  entropy_value
        perecent_left = len(left)/len(parent)
        perecent_right = len(right) / len(parent)
        return entropy(parent)-((perecent_left*entropy(left))+(perecent_right*entropy(right)))



    def _best_split(self, X, y):
        best_split = {}
        best_info_gain = -1
        n_rows, n_cols = X.shape
        for f_idx in range(n_cols):
            X_curr = X[:, f_idx]
            thresholds = np.unique(X_curr)
            for threshold in thresholds:
                df_left = np.concatenate((X[X_curr <= threshold], y[X_curr <= threshold].reshape(-1, 1)), axis=1)
                df_right = np.concatenate((X[X_curr > threshold], y[X_curr > threshold].reshape(-1, 1)), axis=1)
                if (len(df_left) > 0 and len(df_right) > 0):
                    y_left = df_left[:, -1]
                    y_right = df_right[:, -1]
                    gain = self._information_gain(y, y_left, y_right)
                    if gain > best_info_gain:
                        best_info_gain = gain
                        best_split = {
                            'feature_index': f_idx,
                            'threshold': threshold,
                            'df_left': df_left,
                            'df_right': df_right,
                            'gain': gain,
                        }
        return best_split


    def _build(self, X, y, depth=0):
        n_rows, n_cols = X.shape
        if n_rows >= self.min_samples_split and depth <= self.max_depth:
            best = self._best_split(X, y)
            if best['gain'] > 0:
                children = []
                for data, side in [(best['df_left'], 'left'), (best['df_right'], 'right')]:
                    child = self._build(
                        X=data[:, :-1],
                        y=data[:, -1],
                        depth=depth + 1
                    )
                    children.append((side, child))
                return Node(
                    feature=best['feature_index'],
                    threshold=best['threshold'],
                    children=children,
                    gain=best['gain']
                )
        return Node(
            value=Counter(y).most_common(1)[0][0]
        )

    def fit(self, X, y):
        self.root = self._build(X, y)

    def _predict(self, x, tree):
        if tree.value is not None:
            return tree.value
        feature_value = x[tree.feature]
        for side, child in tree.children:
            if side == 'left' and feature_value <= tree.threshold:
                return self._predict(x, child)
            elif side == 'right' and feature_value > tree.threshold:
                return self._predict(x, child)

    def predict(self, X):
        return [self._predict(x, self.root) for x in X]


In [5]:
data = pd.read_csv('onlinefraud.csv')
data.head()

step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  
0  M1979787155             0.0             0.0        0  
1  M2044282225             0.0             0.0        0  
2   C553264065             0.0             0.0        1  
3    C38997010         21182.0             0.0        1  
4  M1230701703             0.0             0.0        0

In [6]:
from sklearn.preprocessing import LabelEncoder
columns_to_convert = ['type','amount','nameOrig', 'oldbalanceOrg','nameDest' , 'oldbalanceDest','newbalanceDest']
label_encoder = LabelEncoder()

for column in columns_to_convert:
    data[column] = label_encoder.fit_transform(data[column])

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
X = data.iloc[:1000, :-1]
y = data.iloc[:1000, -1]
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
X_train, X_test, y_train, y_test = train_test_split(X_imputed.values, y.values, test_size=0.2, random_state=42)
sk_model = DecisionTreeClassifier()
sk_model.fit(X_train, y_train)
sk_preds = sk_model.predict(X_test)

accuracy_score(y_test, sk_preds)


0.945

In [8]:
model = DecisionTree2()
model.fit(X_train, y_train)
preds = model.predict(X_test)
accuracy_score(y_test, preds)

0.96

In [9]:
model = DecisionTree()
model.fit(X_train, y_train)
preds = model.predict(X_test)
accuracy_score(y_test, preds)

0.975

In [10]:
import graphviz

def visualize_tree_graphviz(node, dot=None):
    if dot is None:
        dot = graphviz.Digraph()

    if node.value is not None:
        dot.node(str(id(node)), label=f"Predict: {node.value}")
        return dot

    dot.node(str(id(node)), label=f"Feature {node.feature} <= {node.threshold}\nGain: {node.gain}")
    for side, child in node.children:
        child_dot = visualize_tree_graphviz(child, dot)
        dot.edge(str(id(node)), str(id(child)), label=side)

    return dot
decision_tree = DecisionTree2()
decision_tree.fit(X_train, y_train)
dot = visualize_tree_graphviz(decision_tree.root)
dot.render("decision_tree", format="png", cleanup=True)



'decision_tree.png'